如果觉得这个数据探索有用，请动动小手给我点个赞😜

这个notebook来自于两个公开的notebook

https://www.kaggle.com/holoong9291/eda-for-m5-en

https://www.kaggle.com/artgor/it-is-time-for-m5-going-step-by-step

## M5 competition

此竞赛是预测竞赛，我们需要对未来28天的销售做出正确的预测。 并且，不仅要预测公共榜单上28天的销售数据，还要预测私有榜单上28天的销售数据，分数各占50%。如果在截止日期前还剩一个月，公共榜单将会关闭，开启私有榜单，并且会给出公共榜单上28天的销售数据的真实标签。


并且对于这个数据集有两个比赛，只是要预测的指标不同

* 在这个比赛中我们采用WRMSSE误差 (加权的均方根标准误差);
![](https://i.imgur.com/uqhsf3d.png)
![](https://i.imgur.com/B1hglCf.png)





* 在[另一场比赛](https://www.kaggle.com/c/m5-forecasting-uncertainty)中，我们预测的是商品价格的不确定性分布，采用（WSPL）指标
![](https://i.imgur.com/J8XAQP4.png)
![](https://i.imgur.com/jzLckus.png)
![](https://i.imgur.com/3ihaSZO.png)





![](https://i.imgur.com/C5hASXe.png)

*Work in progress :)*

## 引入工具包

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error, f1_score
pd.options.display.precision = 15
from collections import defaultdict
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import time
from collections import Counter
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from bayes_opt import BayesianOptimization
# import eli5
import shap
from IPython.display import HTML
import json
import altair as alt
from category_encoders.ordinal import OrdinalEncoder
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from typing import List

import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics
from typing import Any
from itertools import product
pd.set_option('max_rows', 500)
import re
from tqdm import tqdm
from joblib import Parallel, delayed

## 载入文件

In [ ]:
path = '/kaggle/input/m5-forecasting-accuracy'
train_sales = pd.read_csv(f'{path}/sales_train_validation.csv')
calendar = pd.read_csv(f'{path}/calendar.csv')
submission = pd.read_csv(f'{path}/sample_submission.csv')
sell_prices = pd.read_csv(f'{path}/sell_prices.csv')

# calender

** calender 各项数据的解释如下:

calender主要包含的是日期数据

date：从2011年1月29日——2016年6月19日的所有日期

wm_yr_wk:wm是WalMart的意思，标号为1，无意义。第2,3位数字代表年份，最后两位数字代表是今年的第多少周，如11101表示日期处在2011年的第一周，


d：对于日期的编号

event_name_1:在那一天发生的第一场大事件

event_type_1:在那一天发生的第一场大事件类型

event_name_2:在那一天发生的第二场大事件

event_type_2:在那一天发生的第二场大事件的类型


snap_CA:加州的商店是否可以在那一天可以使用福利券

snap_TX:得克萨斯州的商店是否可以在那一天使用福利券	

snap_WI:威斯康星州的商店是否可以在那一天使用福利券	






In [ ]:
calendar

# train_sale

**** `train_sale` 各项数据的解释如下: 

item_id:第一个表示商品类别，‘HOBBIES_1_001’表示此项目属于“ HOBBIES”类别，在“ HOBBIES_1”部门出售，编号为001

dept_id：即department_id,表示商品在哪个部门出售

cat_id:即category_id,表示商品的类别

store_id:表示商品在哪个商店出售，CA_1表示该商品在加州的1商店出售

state_id：表示商店所处的州


d_1至d_1913列：表示在第一天至第1913天，每天的商品销售量







In [ ]:
train_sales

# sell_prices

**** `sell_prices` 各项数据的解释如下: 

store_id:表示商店的ID

item_id：表示商品的编号

wm_yr_wk:第2,3位数字代表年份，最后两位数字代表是今年的第多少周，如11325表示日期处在2013年的第25周

sell_price：表示销售价格

In [ ]:
sell_prices

# submission

在`submission`中主要填入对未来28天的预测值，在公共榜单上应该是d_1914天至d_1941天的商品销量预测值

当公共榜单关闭后，将给出d_1914至d_1941天的真实值，然后我们将要预测的则应该是d_1942天至d_1969天商品的销量预测值

In [ ]:
submission

## 进行数据探索

### 一个单独的商品的所有信息

Let's start with looking at all the data for a single item.

先看一看商品编号为`HOBBIES_1_001`商品的信息

In [ ]:
train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001']

我们可以看到，该商品在3个州的10家商店中均有销售。 此项目属于“ HOBBIES”类别，在“ HOBBIES_1”部门出售。

现在来看一看`HOBBIES_1_001`商品的价格

In [ ]:
sell_prices.loc[sell_prices['item_id'] == 'HOBBIES_1_001']

作图来查看一下`HOBBIES_1_001`在第d_1天至d_1913天的销量情况

In [ ]:
plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 6:].values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 5]);
plt.title('HOBBIES_1_001 sales')
plt.legend();

可以看出，该商品在前些时期的销量为零，在第900多天时才有销量

重新作图，让数据看起来更加直观

In [ ]:
plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 6:].rolling(30).mean().values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 5]);
plt.title('HOBBIES_1_001 sales, rolling mean 30 days')
plt.legend();

plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 6:].rolling(60).mean().values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 5]);
plt.title('HOBBIES_1_001 sales, rolling mean 60 days')
plt.legend();

plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 6:].rolling(90).mean().values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001'].iloc[i, 5]);
plt.title('HOBBIES_1_001 sales, rolling mean 90 days')
plt.legend();


So what do we see here?
* there is a definite seasonality with several peaks;
* the sales are more or less constant and quite low (max sales per day in one store is 11);
* as a result, it could be difficult to predict such low values;

In [ ]:
item_prices = sell_prices.loc[sell_prices['item_id'] == 'HOBBIES_1_001']
for s in item_prices['store_id'].unique():
    small_df = item_prices.loc[item_prices['store_id'] == s]
    plt.plot(small_df['wm_yr_wk'], small_df['sell_price'], label=s)
plt.legend()
plt.title('HOBBIES_1_001 sell prices');

除了有一次偶尔的降价，大部分时间价格是保持平稳状态

### 探索某一个商店的所有信息



探索商店CA_1的所有信息

In [ ]:
train_sales.loc[train_sales['store_id'] == 'CA_1']

从表格中，我们可以看到总共有3049种商品在CA_1出售

查看该商店销售的所有商品的价格

In [ ]:
sell_prices.loc[sell_prices['store_id'] == 'CA_1']

查看每个类别以及部门有多少种商品

In [ ]:
ca_1_sales = train_sales.loc[train_sales['store_id'] == 'CA_1']
pd.crosstab(ca_1_sales['cat_id'], ca_1_sales['dept_id'])

这家商店分为3类：食品，爱好和家庭，有2-3个部门。

In [ ]:
plt.figure(figsize=(12, 4))
for d in ca_1_sales['dept_id'].unique():
    store_sales = ca_1_sales.loc[ca_1_sales['dept_id'] == d]
    store_sales.iloc[:, 6:].sum().rolling(30).mean().plot(label=d)
plt.title('CA_1 sales by department, rolling mean 30 days')
plt.legend(loc=(1.0, 0.5));

可以看到 `FOODS_1` 的销量更高.

In [ ]:
item_prices = sell_prices.loc[sell_prices['item_id'] == 'HOBBIES_1_001']
for s in item_prices['store_id'].unique():
    small_df = item_prices.loc[item_prices['store_id'] == s]
    plt.plot(small_df['wm_yr_wk'], small_df['sell_price'], label=s)
plt.legend()
plt.title('HOBBIES_1_001 sell prices');

In [ ]:
ca_1_prices = sell_prices.loc[sell_prices['store_id'] == 'CA_1']
ca_1_prices['dept_id'] = ca_1_prices['item_id'].apply(lambda x: x[:-4])


查看该商店所有类别的销售价格

In [ ]:
plt.figure(figsize=(12, 6))
for d in ca_1_prices['dept_id'].unique():
    small_df = ca_1_prices.loc[ca_1_prices['dept_id'] == d]
    grouped = small_df.groupby(['wm_yr_wk'])['sell_price'].mean()
    plt.plot(grouped.index, grouped.values, label=d)
plt.legend(loc=(1.0, 0.5))
plt.title('CA_1 mean sell prices by dept');

我们可以看到 `HOBBIES_1` 的价格随时间是增长的, 而其他类的价格基本保持平稳.

### 某个部门所含有的所有信息

现在我们可以分析同一部门的不同物品在不同商店的销售情况。

In [ ]:
train_sales.loc[train_sales['dept_id'] == 'HOBBIES_1']

In [ ]:
train_sales.loc[train_sales['dept_id'] == 'HOBBIES_1', 'item_id'].nunique()

我们可以看到该部门有416个商品，并且在所有商店中都出售。

In [ ]:
sell_prices.loc[sell_prices['item_id'].str.contains('HOBBIES_1')]

In [ ]:
hobbies_1_sales = train_sales.loc[train_sales['dept_id'] == 'HOBBIES_1']
plt.figure(figsize=(12, 6))
for d in hobbies_1_sales['store_id'].unique():
    store_sales = hobbies_1_sales.loc[hobbies_1_sales['store_id'] == d]
    store_sales.iloc[:, 6:].sum().rolling(30).mean().plot(label=d)
plt.title('HOBBIES_1 sales by stores, rolling mean 30 days')
plt.legend(loc=(1.0, 0.5));

我们可以看到随着时间的推移销售量在一直会增加。 而且我们可以看到CA_1和CA_3商店的销售额高于其他商店。

In [ ]:
sell_prices.head()

In [ ]:
hobbies_1_prices = sell_prices.loc[sell_prices['item_id'].str.contains('HOBBIES_1')]
plt.figure(figsize=(12, 6))
for d in hobbies_1_prices['store_id'].unique():
    small_df = hobbies_1_prices.loc[hobbies_1_prices['store_id'] == d]
    grouped = small_df.groupby(['wm_yr_wk'])['sell_price'].mean()
    plt.plot(grouped.index, grouped.values, label=d)
plt.legend(loc=(1.0, 0.5))
plt.title('HOBBIES_1 mean sell prices by store');

可以看到销售价格一直在上涨，并且是周期性的增长，从图中可以看出貌似是在某个特殊的时间节点开始增长。

# 总结

* 总共有10家门店分布在三个州

* 共有从2011年1月29日至2016年4月25日，即d_1至d_1913天的所有信息，包括销量

* 有2016年4月25日至2016年6月19日，即d_1914至d_1969天的所有信息，但是不包含销量

* 我们要预测的则是总共60979种商品在d_1914至d_1969总共56天的销量

